In [1]:
import pymongo
import re

client = pymongo.MongoClient(host='localhost', port=27017)
sina = client['Sina']
comments = sina['Comments']

In [2]:
#s_at = '回复@*@重光精神病院保安铁胆*@:@*@*@*@*@*@*@*@*@*@*@*@*@*@*@*@*@共青团中央*@*@*@*@*@*@*@*@*@*@*@*@*@*@*@*@  @*@*@*@*@*@*@*@*@*@*@*@*@*@*@*@*@共青团中央*@*@*@*@*@*@*@*@*@*@*@*@*@*@*@*@'
#print(re.findall('[^回复@]@\*(\S+?)\*@', s_at))

s_emoji = '[汉堡][汉堡]。可以了[台球]'
print(re.findall('\[\S*?\]', s_emoji))

['[汉堡]', '[汉堡]', '[台球]']


In [3]:
#emoji unicode: https://apps.timwhitlock.info/emoji/tables/unicode
import regex
import emoji
flags = [
    b'\xF0\x9F\x87\xA9\xF0\x9F\x87\xAA',
    b'\xF0\x9F\x87\xAC\xF0\x9F\x87\xA7',
    b'\xF0\x9F\x87\xA8\xF0\x9F\x87\xB3',
    b'\xF0\x9F\x87\xAF\xF0\x9F\x87\xB5',
    b'\xF0\x9F\x87\xAB\xF0\x9F\x87\xB7',
    b'\xF0\x9F\x87\xB0\xF0\x9F\x87\xB7',
    b'\xF0\x9F\x87\xAA\xF0\x9F\x87\xB8',
    b'\xF0\x9F\x87\xAE\xF0\x9F\x87\xB9',
    b'\xF0\x9F\x87\xB7\xF0\x9F\x87\xBA',
    b'\xF0\x9F\x87\xBA\xF0\x9F\x87\xB8',
]
flags = [f.decode() for f in flags]  #convert to string


def count_emoji(text, flags):
    #Based on https://stackoverflow.com/a/49242754
    #flags unicode: https://apps.timwhitlock.info/emoji/tables/unicode

    emoji_count = {}
    #find flags first
    for f in flags:
        if text.count(f) > 0:
            emoji_count[f] = text.count(f)
            #emoji_count['contain'] = 1

    #find single unicode emoji
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            if word in emoji_count:
                emoji_count[word] += 1
            else:
                emoji_count[word] = 1
            #emoji_count['contain'] = 1
    return emoji_count


def merge_two_dicts(x, y):
    return {k: x.get(k, 0) + y.get(k, 0) for k in set(x) | set(y)}

In [4]:
print(b'\xF0\x9F\x87\xA8\xF0\x9F\x87\xB3'.decode())
emoji_list = count_emoji("🤔 🙈 me así, se 😌 ds 💕👭👙 hello 👩🏾‍🎓 emoji hello 👨‍👩‍👦‍👦 how are 😊 you today🙅🏽🙅🏽",flags)
print(emoji_list)

🇨🇳
{'🤔': 1, '🙈': 1, '😌': 1, '💕': 1, '👭': 1, '👙': 1, '👩🏾\u200d🎓': 1, '👨\u200d👩\u200d👦\u200d👦': 1, '😊': 1, '🙅🏽': 2}


In [6]:
query_at = {"content": {"$regex": "@\*(\S*)\*@"}}

comments_with_at = comments.find(query_at)
print(comments.find(query_at).count())
at_count = {}
for cmt in comments_with_at:
    at_list = re.findall('[^回复@]@\*(@[^*@]+)\*@', cmt['content'])

    for at in at_list:
        if at != '@':
            if at in at_count:
                at_count[at] += 1
            else:
                at_count[at] = 1

at_count_sort = sorted(at_count.items(), key=lambda item: int(item[1]), reverse=True)
print(at_count_sort[:100])

1269516
[('@带带大师兄', 10754), ('@蔡徐坤', 10363), ('@共青团中央', 2315), ('@紫光阁', 1841), ('@Mr_凡先生', 1802), ('@查看图片', 934), ('@首都网警', 866), ('@平安成都', 844), ('@我都想笑了', 843), ('@人民日报', 793), ('@嗯扫', 774), ('@释明空法师', 773), ('@嗯冲丶', 677), ('@edc陳冠希', 581), ('@出处君', 522), ('@六小龄童', 503), ('@蔡徐坤女粉', 502), ('@新华社中国网事', 490), ('@基督中文', 458), ('@中国反邪教', 442), ('@吴亦凡', 441), ('@带带大开罗', 376), ('@平安中国', 373), ('@观察者网', 355), ('@国家网信办举报中心', 317), ('@央视新闻', 308), ('@平安', 278), ('@edc599', 260), ('@李老八已婚', 245), ('@平安北京', 237), ('@中华女德', 237), ('@清华大学', 234), ('@RestlessBreeze', 232), ('@菜籽油反黑组', 229), ('@平安击毙', 227), ('@观音桥毕加索', 212), ('@我在你的黑名单呢', 211), ('@四周垒起围墙', 204), ('@北京大学', 193), ('@平安日本', 185), ('@-妖_妖-', 185), ('@冯提莫', 168), ('@邋遢大王钲超', 157), ('@我能不能别睡了', 157), ('@蔡徐坤的奶瓶', 150), ('@三分青年儿', 148), ('@李漂亮是你的倾国倾城小猫咪啊', 142), ('@小麻扣', 141), ('@薛之谦', 140), ('@M鹿M', 136), ('@蔡徐坤全球粉丝后援会', 135), ('@平安新津', 133), ('@王思聪', 131), ('@瞎蹦跶乱跳脚', 131), ('@演的你亲妈不认识', 131), ('@清华带学', 125), ('@药水哥ss', 120), ('@扫黄打非', 11

In [7]:
query_emoji_with_bracket = {"content": {"$regex": "\[\S*?\]"}}

comments_with_emoji_with_bracket = comments.find(query_emoji_with_bracket)
print(comments.find(query_emoji_with_bracket).count())

emoji_with_bracket_count = {}
for cmt in comments_with_emoji_with_bracket:
    emoji_with_bracket_list = re.findall('\[\S*?\]', cmt['content'])

    for emoji_with_bracket in emoji_with_bracket_list:
        if emoji_with_bracket in emoji_with_bracket_count:
            emoji_with_bracket_count[emoji_with_bracket] += 1
        else:
            emoji_with_bracket_count[emoji_with_bracket] = 1


emoji_with_bracket_count_sort = sorted(emoji_with_bracket_count.items(), key=lambda item: int(item[1]), reverse=True)
print(emoji_with_bracket_count_sort[:100])

399061
[('[马头]', 113430), ('[狗]', 67205), ('[赛马]', 45249), ('[啤酒]', 44673), ('[爷爷]', 23753), ('[毛毛虫]', 20341), ('[强]', 19579), ('[拍卖]', 13586), ('[8]', 12304), ('[婴儿]', 11694), ('[母鸡]', 11633), ('[看看]', 11627), ('[火焰]', 9517), ('[手掌]', 8590), ('[日本]', 8122), ('[爱心]', 7854), ('[猫咪]', 7522), ('[奶牛]', 7053), ('[奖杯]', 6123), ('[大便]', 5668), ('[汉堡]', 5465), ('[盖饭]', 5288), ('[张嘴]', 4485), ('[幽灵]', 4430), ('[台球]', 4384), ('[小鸟]', 4327), ('[打针]', 4129), ('[卡拉ok]', 4119), ('[热带鱼]', 3819), ('[祈祷]', 3775), ('[家]', 3661), ('[药丸]', 3648), ('[中国]', 3542), ('[鱼]', 3498), ('[足球]', 3155), ('[太開心]', 3054), ('[无]', 3009), ('[猴子]', 2923), ('[青蛙]', 2923), ('[熊]', 2886), ('[4]', 2853), ('[1]', 2641), ('[奶奶]', 2622), ('[篮球]', 2564), ('[老鼠]', 2247), ('[有]', 2167), ('[男孩]', 2117), ('[樱花]', 2076), ('[红中]', 1991), ('[大象]', 1965), ('[苹果]', 1929), ('[大叔]', 1919), ('[蛇]', 1770), ('[卡门]', 1756), ('[老虎]', 1629), ('[德国]', 1616), ('[美国]', 1484), ('[绿茶]', 1454), ('[星星]', 1451), ('[鼻子]', 1354), ('[阿姨]', 1286), ('[得]', 1

In [8]:
comments_with_emoji = comments.find()
emoji_count={}

for cmt in comments.find():
    emoji_count=merge_two_dicts(emoji_count, count_emoji(cmt['content'],flags))
    
emoji_count_sort = sorted(emoji_count.items(), key=lambda item: int(item[1]), reverse=True)
print(emoji_count_sort[:100])

[('🌶️', 9567), ('🍋', 8322), ('🐲', 7271), ('🌶', 5444), ('🐢', 5161), ('🐂', 5128), ('💧', 4215), ('🐝', 3584), ('👅', 3427), ('⭐', 1639), ('❓', 1576), ('🍐', 1539), ('🚪', 1358), ('🤓', 1291), ('⚪', 1144), ('🙋', 1029), ('➕', 992), ('✝️', 966), ('🐈', 898), ('🔪', 872), ('🐉', 855), ('♂️', 850), ('🦐', 812), ('♂', 779), ('🦄', 718), ('🔟', 592), ('🎣', 570), ('♀️', 564), ('🌞', 546), ('🤗', 527), ('🙃', 483), ('\U0001f92e', 481), ('🍹', 470), ('❄️', 439), ('🉑️', 417), ('🍯', 410), ('🐜', 383), ('🦆', 368), ('👁', 350), ('💥', 329), ('🤺', 322), ('⚪️', 320), ('⭐️', 317), ('🕊', 297), ('🥚', 292), ('🐓', 287), ('🌲', 279), ('🔘', 277), ('🕯️', 273), ('🦄️', 257), ('🕯', 240), ('💯', 236), ('🏇', 236), ('👬', 231), ('🕳️', 228), ('🎹', 225), ('👞', 225), ('🙈', 222), ('🦍', 222), ('👓', 221), ('🦅', 215), ('👪', 214), ('🦈', 210), ('☠️', 204), ('💲', 185), ('❄', 180), ('🤢', 180), ('🖐', 179), ('🌳', 172), ('🐊', 170), ('🐏', 170), ('🛏️', 169), ('♀', 166), ('🙊', 163), ('🦁', 161), ('📜', 159), ('🍬', 158), ('👖', 157), ('🍌', 151), ('🌧️', 151), 

In [9]:
emoji_count_sort[1][0].encode()

b'\xf0\x9f\x8d\x8b'